In [10]:
import os
from pywhispercpp.model import Model

In [2]:
model = Model(model='large-v3', models_dir='./whisper.cpp/models')

whisper_init_from_file_with_params_no_state: loading model from '/home/quentin/dev/podcast_transcription/whisper.cpp/models/ggml-large-v3.bin'
whisper_init_with_params_no_state: use gpu    = 1
whisper_init_with_params_no_state: flash attn = 0
whisper_init_with_params_no_state: gpu_device = 0
whisper_init_with_params_no_state: dtw        = 0
ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060 Laptop GPU, compute capability 8.6, VMM: yes
whisper_init_with_params_no_state: devices    = 2
whisper_init_with_params_no_state: backends   = 2
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51866
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 1280
whisper_model_load: n_audio_head  = 20
whisper_model_load: n_audio_layer = 32
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 1280
whisper_model_load: n_text_head   =

In [38]:
file = './s10e43_evenMoreTrimmed_benchmark.mp3'

def gathering_files_in_directory() -> list[str]:
    print('test')

def transcribe(file:str):
    transcription = model.transcribe(
        file, 
        language='fr',
        temperature=0.0,
        print_progress=True,
        # print_confidence=True
    )
    return transcription

In [40]:
def transcribe_into_file(file:str):

    # directory to store the transcriptions into 
    output_directory = 'transcriptions_output'
    os.makedirs(f'./{output_directory}', exist_ok=True)

    # name of the transcription output file
    # (using the name of the file given as params)
    base_name = os.path.splitext(file)[0]
    output_name = f"{base_name}.txt"

    # writing transcriptions in file
    with open(f'./{output_directory}/{output_name}','w',encoding='utf-8') as output_file:
        for segment in transcribe(file):
            output_file.write(f'{segment}')
            print(segment)


def format_output():
    print('format')

def store_into_db():
    print('db')

In [41]:
transcribe_into_file(file)

# transcribe(file)

# unload model from vram

Progress:   0%
Progress:  18%
Progress:  38%
Progress:  59%
Progress:  80%


t0=0, t1=140, text=Message publicitaire.
t0=140, t1=1022, text=Le médicament Daflon 500 mg, indiqué pour soulager les jambes lourdes et douloureuses dues à l'insuffisance veineuse, agit en renforçant le tonus veineux et en protégeant les vaisseaux sanguins.
t0=1022, t1=1872, text=Son efficacité a été cliniquement démontrée dans le traitement des troubles de la circulation veineuse, jambes lourdes, douleurs, impatience, en complément des mesures hygiéno-diététiques.
t0=1872, t1=2034, text=Rendez-vous sur Daflon.fr.
t0=2034, t1=2624, text=Daflon 500 mg, composé de fractions flavonoïques purifiées micronisées et réservées à l'adulte, est disponible en pharmacie sans ordonnance.
t0=2624, t1=3006, text=Tout médicament peut exposer à des risques. Parlez-en à votre pharmacien et lisez attentivement la notice.
t0=3006, t1=3230, text=Si les symptômes persistent, consultez votre médecin.
t0=5624, t1=6090, text=Bonjour, je suis là. Vous qui n'avez pas d'invité pour nous mettre la honte de cette i

Progress: 100%
